<a href="https://colab.research.google.com/github/betsyvies/food-sales-predictions/blob/main/predictive_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn import set_config
set_config(display='diagram')

In [4]:
path = '/content/drive/MyDrive/Data science/Projects/Project 1/sales_predictions_2023.csv'
df = pd.read_csv(path)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


### Explorar los datos

Tenemos columnas con características categoricas, nominales y ordinales, comó tambien numéricas con datos enteros y flotantes. Nos faltaría explorar los datos para saber que columnas tienen datos faltantes y sus tipos de datos.

Puedo ver que son 2 las columnas con datos faltantes estás son de tipo object y flotante. La columna de tipo entero está completa.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [6]:
df['Outlet_Size'].isna().sum()

2410

### División de la validación
Separaré mi columna objetivo ***Item_Outlet_Sales*** de las demás columnas, también eliminaré mi columna objetivo junto con las columnas que no aportaran datos de valor al modelo, para este caso serian los identificadores ***Item_Identifier*** y ***Outlet_Identifier***. Las columnas que queden en el DataFrame serán entrenadas.

Luego haré la división de los datos en datos de prueba y entrenamiento.

In [7]:
X = df.drop(['Item_Outlet_Sales', 'Item_Identifier', 'Outlet_Identifier'], axis=1)
y = df['Item_Outlet_Sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

### Arreglo de cualquier categoría inconsistente en los datos

En el desarrollo exploratorio pude notar que la columna ***item_fat_content*** tiene inconsistencia en los datos, haré el cambio de estos.

In [8]:
X_train['Item_Fat_Content'].value_counts()

Item_Fat_Content
Low Fat    3783
Regular    2176
LF          260
reg          87
low fat      86
Name: count, dtype: int64

Hay tres maneras de nombrar low fat y dos de regular, para que estos valores sean consistentes los remplazaré por 'Low Fat' y 'Regular', ya que serán los datos con los que decido trabajar en esta columna.

In [9]:
X_train['Item_Fat_Content'] = X_train['Item_Fat_Content'].replace(['LF', 'low fat', 'reg'], ['Low Fat', 'Low Fat', 'Regular'])
X_test['Item_Fat_Content'] = X_test['Item_Fat_Content'].replace(['LF', 'low fat', 'reg'], ['Low Fat', 'Low Fat', 'Regular'])

Al volver hacer el conteo puedo observar que solo hay dos categorias en ambos data sets (train y test), lo cual le dará mayor consistencia a mi análisis.

In [10]:
X_train['Item_Fat_Content'].value_counts()

Item_Fat_Content
Low Fat    4129
Regular    2263
Name: count, dtype: int64

In [11]:
X_test['Item_Fat_Content'].value_counts()

Item_Fat_Content
Low Fat    1388
Regular     743
Name: count, dtype: int64

### Instanciar selectores de columnas
Definiré los selectores para las columnas con valor object y otra para la columna con valor number. Lo que permitirá que el código siga funcionando en producción incluso si las columnas del DataFrame cambian.

In [12]:
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

### Instanciar transformadores
Usaré tres diferentes transformadores: SimpleImputer, StandardScaler y OneHotEncoder. También, instanciaré dos SimpleImputers con diferentes estrategias de imputación para los valores faltantes: most_frequent y mean. El primero para los categóricos y el segundo para los númericos.

In [13]:
# Imputers
freq_imputer = SimpleImputer(strategy='most_frequent')
mean_imputer = SimpleImputer(strategy='mean')
# Scaler
scaler = StandardScaler()
# One-hot encoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

### Instanciar pipelines

Definiré dos pipelines, uno para los datos numéricos y otros para los datos nominales categóricos. Pasandole al pipeline númerico el mean_imputer y el scaler, y al pipeline categórico el freq_imputer y el one-hot encoder.

In [14]:
# Numeric pipeline
numeric_pipe = make_pipeline(mean_imputer, scaler)
numeric_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

In [15]:
# Categorical pipeline
categorical_pipe = make_pipeline(freq_imputer, ohe)
categorical_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

### Instanciar ColumnTransformer

Ahora crearé 2 tuplas, estás tendrán como primer valor el pipeline y como segundo valor el selector. Para el pipeline númerico el selector númerico y para el categorico el selector categorico. La función make_column_transformer utiliza tuplas para hacer coincidir los transformadores con los tipos de datos sobre los que deben actuar.

In [16]:
# Tuples para Column Transformer
number_tuple = (numeric_pipe, num_selector)
category_tuple = (categorical_pipe, cat_selector)
# ColumnTransformer
preprocessor = make_column_transformer(number_tuple, category_tuple)
preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x78ac1f696920>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x78ac1f6945b0>)])

### Instanciar y ajustar el transformador en los datos de entrenamiento
Ajustare el **preprocessor** que es un ColumnTransformer solo en los datos de entrenamiento. El cual no se debe aplicar a los datos de prueba. Esto para que todos los cálculos del escalamiento solo se basen en los datos de entrenamiento.

In [17]:
# fit on train
preprocessor.fit(X_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x78ac1f696920>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x78ac1f6945b0>)])

Usaré este ColumnTransformer ajustado para transformar los conjuntos de datos de entrenamiento y de prueba.

In [18]:
# transform train and test
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

### Inspeccionar el resultado
Al inspeccionar el resultado me aseguro de que se hayan sustituido los datos faltantes, que los datos categóricos hayan sido codificados con one-hot y que los datos numéricos se hayan escalado.

1. Comprobar que no hay datos faltantes

In [19]:
print(np.isnan(X_train_processed).sum().sum(), 'missing values in training data')
print(np.isnan(X_test_processed).sum().sum(), 'missing values in testing data')

0 missing values in training data
0 missing values in testing data


2. Revisar el tipo de dato de los datos de prueba y entrenamiento

In [20]:
print('All data in X_train_processed are', X_train_processed.dtype)
print('All data in X_test_processed are', X_test_processed.dtype)

All data in X_train_processed are float64
All data in X_test_processed are float64


3. Comprobar que todos los datos númericos fueron escalados y que los categóricos tengan codificación one-hot encoder.

In [21]:
# Array NumPy de los datos de entrenamiento
X_train_processed

array([[ 0.81724868, -0.71277507,  1.82810922, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.5563395 , -1.29105225,  0.60336888, ...,  0.        ,
         1.        ,  0.        ],
       [-0.13151196,  1.81331864,  0.24454056, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [ 1.11373638, -0.92052713,  1.52302674, ...,  1.        ,
         0.        ,  0.        ],
       [ 1.76600931, -0.2277552 , -0.38377708, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.81724868, -0.95867683, -0.73836105, ...,  1.        ,
         0.        ,  0.        ]])

In [22]:
# Array NumPy de los datos de prueba
X_test_processed

array([[ 0.33100885, -0.77664625, -0.99881554, ...,  1.        ,
         0.        ,  0.        ],
       [-1.17989246,  0.1003166 , -1.58519423, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.37844688, -0.48299432, -1.59578435, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [-1.13957013,  1.21832428,  1.09397975, ...,  1.        ,
         0.        ,  0.        ],
       [-1.49772727, -0.77809567, -0.36679966, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.52076098, -0.77976293,  0.11221189, ...,  1.        ,
         0.        ,  0.        ]])

## Importar los modelos y las metricas de rendimiento

In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

### Instanciar el modelo de regresión lineal

In [24]:
reg = LinearRegression()

### Entrenar el modelo con los datos de entrenamiento

Hago este paso para que el modelo aprenda sobre la relación entre las características y el objetivo.

El modelo aprenderá la relación entre X e y.

In [25]:
reg.fit(X_train_processed, y_train)

LinearRegression()

### Obtención de predicciones
Para calcular la métrica, necesito extraer las predicciones del modelo y guardarlas como una variable.

In [26]:
train_preds = reg.predict(X_train_processed)
test_preds = reg.predict(X_test_processed)

### Medición del rendimiento del modelo

#### 1. Puntuación R^2.

- R² = 1 indica que el modelo explica perfectamente la variabilidad de los datos.
- R² = 0 indica que el modelo no explica nada de la variabilidad de los datos.
- Valores negativos pueden ocurrir si el modelo es peor que una línea horizontal.

In [27]:
r2_train = r2_score(y_train, train_preds)
r2_test = r2_score(y_test, test_preds)
print(r2_train)
print(r2_test)

0.5611923313774996
0.5670511340874516


#### 2. Raíz del error cuadrático medio (RECM)

Es la raíz cuadrada del ECM y proporciona una medida de la precisión del modelo en las mismas unidades que los datos originales.

- Facilita la interpretación del ECM en las mismas unidades que los datos.
- Cuanto menor sea el RECM, mejor es el rendimiento del modelo.

In [28]:
rmse_train = np.sqrt(mean_squared_error(y_train, train_preds))
rmse_test = np.sqrt(mean_squared_error(y_test, test_preds))
print(rmse_train)
print(rmse_test)

1139.5752972466798
1092.9300470910157


Puedo ver que las metricas de medición R2 y RECM tanto en train como en test tienen valores similares con una mínima diferencia. Lo cual me hace apreciar consistencia en el rendimiento del modelo, sugiriendo que no está ni sobreajustado ni subajustado. De igual forma 0.56 no es el mejor resultado así que probaré con otro modelo para ver si esto se puede mejorar.


### Instanciar el modelo de árbol de regresión

In [29]:
dec_tree = DecisionTreeRegressor(random_state = 42)

### Entrenar el modelo con los datos de entrenamiento

In [30]:
dec_tree.fit(X_train_processed, y_train)

DecisionTreeRegressor(random_state=42)

### Obtención de predicciones

In [31]:
train_preds_dec = dec_tree.predict(X_train_processed)
test_preds_dec = dec_tree.predict(X_test_processed)

### Medición del rendimiento del modelo

#### 1. Puntuación R^2.

In [32]:
r2_train_dec = r2_score(y_train, train_preds_dec)
r2_test_dec = r2_score(y_test, test_preds_dec)
print(r2_train_dec)
print(r2_test_dec)

1.0
0.18054607210496232


#### 2. Raíz del error cuadrático medio (RECM)

In [33]:
rmse_train_dec = np.sqrt(mean_squared_error(y_train, train_preds_dec))
rmse_test_dec = np.sqrt(mean_squared_error(y_test, test_preds_dec))
print(rmse_train_dec)
print(rmse_test_dec)

4.925864104892086e-15
1503.6139050522115


Puedo ver que hay un sobreajuste severo, lo que quiere decir que el modelo está aprendiendo demasiado bien los detalles y el ruido de los datos de entrenamiento, y por lo tanto, no puede generalizar nuevos datos.

###  Ajustar el modelo

In [34]:
# Busquen las opciones para ajustar este modelo
dec_tree.get_params()

{'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 42,
 'splitter': 'best'}

El valor por omisión para max_depth es None, ya que el modelo no estaba limitado. Antes de ajustar este parámetro, revisaré cuál era la profundidad del árbol predeterminado.

In [35]:
dec_tree.get_depth()

40

Puedo ver que mi árbol tiene una profundidad de 40.

También que los nodos hojas de mi árbol son 6265.

In [36]:
dec_tree.get_n_leaves()

6265

### Encontrar el max_depth óptimo (ajuste de hiperparámetro)
En primer lugar, me iré al extremo y probaré con una profundidad máxima de 2 para ver como el cambio de este parámetro afecta a mi modelo.

In [37]:
dec_tree_2 = DecisionTreeRegressor(max_depth = 2, random_state = 42)
dec_tree_2.fit(X_train_processed, y_train)
train_2_score = dec_tree_2.score(X_train_processed, y_train)
test_2_score = dec_tree_2.score(X_test_processed, y_test)
print(train_2_score)
print(test_2_score)

0.43164096170474664
0.4337775044707164


Puedo ver que al cambiar a una profundidad máxima de 2 las metricas mejoran. Ahora probaré con parametros cercanos al 2 para ver si pueden mejorar aún más.

In [38]:
dec_tree_5 = DecisionTreeRegressor(max_depth = 5, random_state = 42)
dec_tree_5.fit(X_train_processed, y_train)
train_5_score = dec_tree_5.score(X_train_processed, y_train)
test_5_score = dec_tree_5.score(X_test_processed, y_test)
print(train_5_score)
print(test_5_score)

0.6039397477322956
0.5947099753159972


In [39]:
train_preds_5 = dec_tree_5.predict(X_train_processed)
test_preds_5 = dec_tree_5.predict(X_test_processed)

rmse_train_dec = np.sqrt(mean_squared_error(y_train, train_preds_5))
rmse_test_dec = np.sqrt(mean_squared_error(y_test, test_preds_5))
print(rmse_train_dec)
print(rmse_test_dec)

1082.6461900869947
1057.4431299496734


Despues de probar con profundidades máximas en un rango de 2 al 7 pude notar que la profundidad máxima con mejor resultado es el 5. Teniendo un valor de **59.47% R2** cercano al obtenido en el entrenamiento, esto indica que el modelo tiene una ***capacidad de generalización bastante buena y no está sobreajustado***. El ***60.4%*** de la variabilidad en los ***datos de entrenamiento***. No es extremadamente alto, pero muestra que el modelo tiene un ajuste decente para los datos de entrenamiento.También puedo ver que los valores de el RECM también son cercanos lo que reafirma que el modelo generalización bastante bien. Un **RMSE de 1057** representa aproximadamente el **8.1% del rango total de 33.00 a 13086.00 dólares**, ya que los márgenes de beneficio son altos y la variabilidad en las ventas es grande, un RMSE del 8.1% del rango total ***podría ser aceptable***.